In [ ]:
!pip install numpy==1.24.3

In [ ]:
!pip install -qU crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 openai

In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from openai import OpenAI
import time
import pandas as pd
from langchain.agents import Tool
from google.colab import userdata
from langchain_openai import ChatOpenAI

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:913: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(


# Connection to LLM

In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get('openai')

class CFG:
    model = 'gpt-4o-mini'
    temp = 0
    client_id = 'demo@demo.com'

llm = ChatOpenAI(model=CFG.model, temperature=CFG.temp)

The code retrieves the OpenAI API key from a userdata dictionary and sets it as an environment variable. It defines a configuration class (CFG) that specifies the model to use (gpt-4o-mini), the temperature (0, for deterministic output), a path to a folder containing PDFs, and a dummy client ID. Finally, it initializes a language model (llm) using the specified model and temperature settings.

In [ ]:
#to check if the model operates
llm.invoke('Hello').content

'Hello! How can I assist you today?'

In [ ]:
#this is some simulation of any external data interaction
product_portfolio = {
    'TV': [
        {'id': '1234', 'product_name': 'TV set Sony', 'price': '300', 'manual': 'http://manual/tvsetSony'}
    ],
    'mobile phone': [
        {'id': '1238', 'product_name': 'Samsung', 'price': '250', 'manual': 'http://manual/mobilesamsung'}
    ]
}

orders_database = {
    'demo@demo.com': [
        {'order_number': '1235', 'product_name': 'Apple', 'price': '350', 'status': 'waiting for confirmation'}
    ]
}


In [ ]:
def search_products(query: str) -> str:
    for category, products in product_portfolio.items():
        for product in products:
            if query.lower() in product["product_name"].lower():
                return f'{product["product_name"]} - ${product["price"]} - Manual: {product["manual"]}'
    return "Product not found."

def check_order_status(_: str) -> str:
    orders = orders_database.get(CFG.client_id, [])
    if not orders:
        return "No orders found."
    return "\n".join([f'{o["product_name"]}: {o["status"]}' for o in orders])

search_tool = Tool(name="ProductSearch", func=search_products, description="Search product info")
order_status_tool = Tool(name="OrderStatus", func=check_order_status, description="Check order status")

search_products
searches the product catalog (product_portfolio) to find a product matching the query.

If found, it returns the product name, price, and a link to the manual.

If not found, it returns "Product not found."

check_order_status checks the order status for the client with the ID CFG.client_id in the orders_database.

If no orders are found, it returns "No orders found."

Otherwise, it returns a list of ordered products along with their statuses.

In [ ]:
#define subagents
purchase_agent = Agent(
    role="Purchase Agent",
    goal="Answer product-related questions using tools and database",
    backstory="Helps customers with product questions and order info.",
    tools=[search_tool, order_status_tool],
    llm=llm,
    verbose=True
)

complaint_agent = Agent(
    role="Complaint Agent",
    goal="Process new complaints or provide status",
    backstory="Handles new complaints and tracks existing ones.",
    tools=[],
    llm=llm,
    verbose=True
)

return_agent = Agent(
    role="Return Agent",
    goal="Handle return requests or status questions",
    backstory="Supports customers in returning products and updates systems.",
    tools=[],
    llm=llm,
    verbose=True
)

other_agent = Agent(
    role="General Support Agent",
    goal="Answer miscellaneous customer questions",
    backstory="Handles general or unclear inquiries and routes them if needed.",
    tools=[],
    llm=llm,
    verbose=True
)

This code defines four specialized AI agents for handling different types of customer service tasks. Each agent is initialized with a specific role, goal, backstory, and access to tools (if needed). Here's a summary:

purchase_agent: Assists customers with product-related questions and order status. It uses tools for product search and checking order information.

complaint_agent: Manages customer complaints, including submitting new ones and checking the status of existing ones. It doesn't use any tools in this setup.

return_agent: Handles product return requests and related questions. Like the complaint agent, it currently has no tools assigned.

other_agent: Takes care of general or unclear customer inquiries and routes them appropriately if needed. It also operates without tools.

In [ ]:
# === CLASSIFIER AGENT ===

classifier_agent = Agent(
    role="Customer Service Manager",
    goal="Analyze customer message and decide intent category",
    backstory="""
    You are responsible for understanding the customer's intent based on the message
    and triggering the correct agent: Purchase, Complaint, Return, or Other.
    """,
    llm=llm,
    verbose=True
)

# === CLASSIFICATION TASK ===

def classify_intent(message: str) -> str:
    classification_prompt = f"""
    You are a classifier. Given the message: "{message}", decide its intent category.
    Choose one of: Purchase, Complaint, Return, Other.
    Only output one word.
    """
    response = llm.invoke(classification_prompt)
    return response.content.strip()

# === DELEGATION LOGIC ===

def delegate_task(message: str):
    category = classify_intent(message)
    print(f"📌 Classified as: {category}")

    if category.lower() == "purchase":
        task = Task(
            description=f"Customer message: '{message}'. Answer using tools.",
            expected_output="Product info or order details.",
            agent=purchase_agent
        )
    elif category.lower() == "complaint":
        task = Task(
            description=f"Customer message: '{message}'. Respond as complaint handler.",
            expected_output="Complaint processed.",
            agent=complaint_agent
        )
    elif category.lower() == "return":
        task = Task(
            description=f"Customer message: '{message}'. Handle return.",
            expected_output="Return steps provided.",
            agent=return_agent
        )
    else:
        task = Task(
            description=f"Customer message: '{message}'. Handle as general inquiry.",
            expected_output="Helpful response or escalation.",
            agent=other_agent
        )

    crew = Crew(
        agents=[task.agent],
        tasks=[task],
        verbose=True
    )
    return crew.kickoff()

classifier_agent

An agent acting as a "Customer Service Manager".

Its role is to understand the customer's message and determine which category it belongs to.

classify_intent(message: str)

A function that sends the customer's message to the language model with a prompt.

The model returns one of four intent categories: Purchase, Complaint, Return, or Other.

delegate_task(message: str)

Uses the classify_intent function to determine the category of the message.

Based on the result, it creates a task for the matching agent (purchase_agent, complaint_agent, return_agent, or other_agent).

It then starts a Crew with the selected agent and task, and runs the conversation using .kickoff().

In [ ]:
# === TEST EXAMPLES ===

test_messages = [
    "I want to know more about the TV set Sony.",
    "I received a broken phone. I want to file a complaint.",
    "I want to return my Apple phone.",
    "What are your support hours?"
]

for msg in test_messages:
    print(f"\n🟦 CUSTOMER MESSAGE: {msg}")
    response = delegate_task(msg)
    print(f"✅ AGENT RESPONSE:\n{response}\n")


🟦 CUSTOMER MESSAGE: I want to know more about the TV set Sony.
📌 Classified as: Other
 [DEBUG]: == Working Agent: General Support Agent
 [INFO]: == Starting Task: Customer message: 'I want to know more about the TV set Sony.'. Handle as general inquiry.


> Entering new CrewAgentExecutor chain...
I now can give a great answer  
Final Answer: Thank you for your inquiry about Sony TV sets! Sony offers a wide range of televisions that cater to various needs and preferences. Their lineup includes OLED, LED, and 4K Ultra HD models, known for their exceptional picture quality, vibrant colors, and advanced technology. 

Some key features of Sony TVs include:

1. **Picture Quality**: Sony's OLED TVs provide deep blacks and stunning contrast, while their LED models offer bright and vivid images. Many models also support HDR (High Dynamic Range) for enhanced color and detail.

2. **Smart Features**: Most Sony TVs come with Android TV, allowing you to access a variety of streaming services, apps

📌 Classified as: Complaint
 [DEBUG]: == Working Agent: Complaint Agent
 [INFO]: == Starting Task: Customer message: 'I received a broken phone. I want to file a complaint.'. Respond as complaint handler.


> Entering new CrewAgentExecutor chain...
I now can give a great answer  
Final Answer: Thank you for reaching out to us regarding the issue with your phone. I’m sorry to hear that you received a broken device. I will assist you in filing a complaint and ensure that it is processed promptly. 

To proceed, I need to gather some information from you:

1. **Order Number**: Please provide the order number associated with your purchase.
2. **Description of the Damage**: Could you describe the specific issues with the phone? For example, is the screen cracked, or is it not turning on?
3. **Photos**: If possible, please attach any photos of the damage to help us assess the situation better.
4. **Preferred Resolution**: What would you like us to do? Would you prefer a replacement, a repair, 

📌 Classified as: Return
 [DEBUG]: == Working Agent: Return Agent
 [INFO]: == Starting Task: Customer message: 'I want to return my Apple phone.'. Handle return.


> Entering new CrewAgentExecutor chain...
I now can give a great answer  
Final Answer: To initiate the return of your Apple phone, please follow these steps:

1. **Check the Return Policy**: Ensure that your purchase is eligible for return. Typically, Apple products can be returned within 14 days of receipt, provided they are in their original condition and packaging.

2. **Prepare the Phone for Return**:
   - Make sure to back up any data you want to keep, as returning the phone will erase all personal information.
   - Remove any accessories, cases, or screen protectors.
   - Include all original packaging, manuals, and any included accessories.

3. **Obtain a Return Authorization**:
   - Visit the Apple website or the retailer's website where you purchased the phone.
   - Log into your account and navigate to the order hi

📌 Classified as: Other
 [DEBUG]: == Working Agent: General Support Agent
 [INFO]: == Starting Task: Customer message: 'What are your support hours?'. Handle as general inquiry.


> Entering new CrewAgentExecutor chain...
I now can give a great answer  
Final Answer: Our support hours are Monday through Friday, from 9:00 AM to 5:00 PM (local time). If you have any inquiries outside of these hours, please feel free to leave a message or send an email, and we will get back to you as soon as possible during our next business day. If you need immediate assistance, please check our FAQ section on our website for common questions and answers. Thank you for reaching out!

> Finished chain.
 [DEBUG]: == [General Support Agent] Task output: Our support hours are Monday through Friday, from 9:00 AM to 5:00 PM (local time). If you have any inquiries outside of these hours, please feel free to leave a message or send an email, and we will get back to you as soon as possible during our next business